In [4]:
!pip install deims
!pip install xmltodict


[notice] A new release of pip is available: 23.3.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.3.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [5]:
import uuid
import json
import xmltodict
import deims
import datetime
import os

In [9]:
with open('getcapabilities_1.3.0.xml') as fd:
    doc = xmltodict.parse(fd.read())

for layer in doc['WMS_Capabilities']['Capability']['Layer']['Layer']:

    # date stamp of metadata update date
    now = datetime.datetime.now()

    # fetch EPSG code
    epsg_code = ''
    for value in layer['CRS']:
        if "EPSG:" in value:
            epsg_code = value[5:]

    # boundaries
    westBoundLongitude = float(layer['EX_GeographicBoundingBox']["westBoundLongitude"])
    eastBoundLongitude = float(layer['EX_GeographicBoundingBox']["eastBoundLongitude"])
    southBoundLatitude = float(layer['EX_GeographicBoundingBox']["southBoundLatitude"])
    northBoundLatitude = float(layer['EX_GeographicBoundingBox']["northBoundLatitude"])

    longitude_centre = (westBoundLongitude+eastBoundLongitude)/2.0
    latitude_centre = (northBoundLatitude+southBoundLatitude)/2.0

    keywords_to_be_printed = ["eLTER"]
    record_uuid = None
    authors = None
    so_reference = None
    # process keywords for site information and temporal extent
    for keyword in layer['KeywordList']['Keyword']:
        uri = None

        if "id: " in keyword:
            record_uuid = keyword[4:]
            continue

        if "project: " in keyword:
            if "e-shape" in keyword:
                project = {
                    "projectName": "e-shape",
                    "projectID": "https://doi.org/10.3030/820852"
                }
                continue

            if "agame" in keyword:
                project = {
                    "projectName": "AGAME",
                    "projectID": "https://www.umweltbundesamt.at/en/services/agame"
                }
                continue

        if "variable: " in keyword:
            data_product_type = keyword[10:]
            if data_product_type == "Phenology":
                keyword = "Phenology"
                uri = "https://vocabs.lter-europe.net/EnvThes/21647"
                creators = [{
                  "creatorGivenName": "Saverio",
                  "creatorFamilyName": "Vicario",
                  "creatorEmail": "saverio.vicario@iia.cnr.it",
                  "creatorIDs": [{
                      "entityID": "https://orcid.org/0000-0003-1140-0483",
                      "entityIDSchema": "Orcid",
                   }]
                }]

            if data_product_type == "Hydroperiod":
                keyword = "Hydroperiod"
                creators = [{
                  "creatorGivenName": "Ioannis",
                  "creatorFamilyName": "Manakos",
                  "creatorEmail": "imanakos@iti.gr",
                  "creatorIDs": [{
                      "entityID": "https://orcid.org/0000-0001-6833-294X",
                      "entityIDSchema": "Orcid",
                    }]
                }]

            if data_product_type == "Gross Primary Production":
                keyword = "Gross Primary Production"
                uri = "http://vocabs.lter-europe.net/EnvThes/21074"
                so_reference = {
                    "datasetTypeCode": "Gross Primary Production",
                    "datasetTypeURI": "https://vocabs.lter-europe.net/so/090"
                }
                creators = [{
                    "creatorGivenName": "Alberto",
                    "creatorFamilyName": "Fuentes-Monjaraz",
                    "creatorEmail": "mario.fuentesmonjaraz@deltares.nl",
                    "creatorIDs": [{
                        "entityID": "https://orcid.org/0000-0002-0711-8867",
                        "entityIDSchema": "Orcid",
                    }]
                },
                {
                    "creatorGivenName": "Anna",
                    "creatorFamilyName": "Spinosa",
                    "creatorEmail": "anna.spinosa@deltares.nl",
                    "creatorIDs": [{
                        "entityID": "https://orcid.org/0000-0002-7921-5922",
                        "entityIDSchema": "Orcid",
                    }]
                },
                {
                  "creatorGivenName": "Valeria",
                  "creatorFamilyName": "Mobilia",
                  "creatorEmail": "valeria.mobilia@deltares.nl",
                  "creatorIDs": [{
                      "entityID": "https://orcid.org/0000-0001-9370-598X",
                      "entityIDSchema": "Orcid",
                    }]
                }]

            if data_product_type == "snowcover":
                uri = "https://vocabs.lter-europe.net/EnvThes/21559"
                keyword = "snowCover"
                creators = [{
                    "creatorGivenName": "Maria",
                    "creatorFamilyName": "Adamo",
                    "creatorEmail": "adamo@iia.cnr.it",
                    "creatorIDs": [{
                        "entityID": "https://orcid.org/0000-0003-3030-4884",
                        "entityIDSchema": "Orcid",
                     }]
                },
                {
                    "creatorGivenName": "Chiara",
                    "creatorFamilyName": "Richiardi",
                    "creatorEmail": "chiara.richiardi@iia.cnr.it",
                    "creatorIDs": [{
                        "entityID": "https://orcid.org/0000-0002-2370-7768",
                        "entityIDSchema": "Orcid",
                     }]
                }]

        if "site: " in keyword:
            deims_suffix = keyword[6:]
            #print(site_record)
            uri = "https://deims.org/" + deims_suffix
            site_record = deims.getSiteById(deims_suffix)

            try:
                site_name = site_record["title"]
            except:
                print(layer['Title'])
                print(deims_suffix)
            continue

        if "https://doi.org/" in keyword:
            doi = {
                "alternateID": keyword,
                "alternateIDType": "DOI"
            }

        if "time: " in keyword:
            time_string = keyword[6:]

            if "-" in time_string:
                time_values = time_string.split('-')
                period = [{
                    "startDate": time_values[0] + '-01-01',
                    "endDate": time_values[1] + '-12-31'
                }]

            else:
                period = [{
                    "startDate": time_string + '-01-01',
                    "endDate": time_string + '-12-31'
                }]
            continue

        if uri:
            formatted_keyword = {
                "keywordLabel": keyword,
                "keywordURI": uri
            }
        else:
            formatted_keyword = {
                "keywordLabel": keyword,
                "keywordURI": ""
            }

        keywords_to_be_printed.append(formatted_keyword)

    #print(deims_suffix)

    record = {
        "metadata": {
            "alternateIdentifiers": [doi],
            "titles": [{
                "titleLanguage": "eng",
                "titleText": layer['Title']
            }],
            "descriptions": [{
                "type": "Abstract",
                "descriptionText": layer['Abstract'] # we could add a sentence that the metadata record was generated automatically
            }],
            "keywords": keywords_to_be_printed,
            "publicationDate": datetime.datetime.now().strftime('%Y-%m-%d'),
            "language": "english",
            "datasetType": so_reference,
            "creators": creators,
            "licenses": [{
                "name": "CC BY 4.0",
                "url": "https://creativecommons.org/licenses/by/4.0/deed.en"
            }],
            "geoLocations": [
                {
                    "geographicDescription": site_record["title"],
                    "point": {
                        "longitude": longitude_centre,
                        "latitude": latitude_centre
                    }
                },
                {
                      "geographicDescription": site_record["title"],
                      "BoundingBox": {
                           "westBoundLongitude": westBoundLongitude,
                           "eastBoundLongitude": eastBoundLongitude,
                           "southBoundLatitude": southBoundLatitude,
                           "northBoundLatitude": northBoundLatitude 
                      }
                 }
            ],
            "temporalCoverages": period,
            "project": project,
            "siteReference": [{
                "name": site_record["title"],
                "PID": deims_suffix
            }],
        },
        "files": {
            "enabled": False
        }
    }

    if project["projectName"] == "AGAME":
        record["metadata"]["methods"]["instrumentationDescription"] = "The methodology integrates multiple data sources via machine learning techniques to estimate Gross Primary Production (GPP) across different ecosystems. The process begins with data pre-processing, including the selection of sites based on criteria such available vegetation information, at least three full years of eddy covariance flux data. GPP and environmental data (e.g. air temperature, vapor pressure deficit, etc.) are extracted from the ICOS database across different ecosystem types. Then, different remote sensing (RS) indices (e.g NDVI, EVI, etc.) are estimated in GEE using Sentinel-2 MSI data as the mean value of the pixels found inside the climatological footprint 70 (an area were 70% of the GPP measurements are coming from). Both data coming from ICOS dataset and RS indices are used as predictors for the model. The data is split, with 70% used for training and 30% for testing. In the model setup, an XGBoost model is trained using the selected environmental and RS based indices. The model parameters are fine-tuned to improve accuracy. The remaining 30% of testing data is used to evaluate the model’s performance by comparing its predictions against in-situ GPP data. Error metrics like Mean Absolute Error (MAE), Root Mean Absolute Error (RMAE), and R² are provided. The maps computation phase applies the trained model to ecosystem boundaries from the DEIMS website to generate 5-day GPP maps."
    
    filename = record_uuid + ".json"
    base_dir = os.getcwd() + '/files_for_catalogue'
    destination_path = os.path.join(base_dir, filename)

    with open(destination_path, "w+") as f:
        json.dump(record, f)


KeyboardInterrupt: 

In [3]:
# to zip all generated files
import shutil
shutil.make_archive("files_for_catalogue", 'zip', base_dir)

'/home/wohner/work/files_for_catalogue.zip'